In [4]:
class ParkingSpace:
    def __init__(self, space_id, base_cost):
        self.space_id = space_id
        self.base_cost = base_cost
        self.cost = base_cost
        self.is_occupied = False

    def occupy(self):
        self.is_occupied = True

    def vacate(self):
        self.is_occupied = False

class EVParkingSpace(ParkingSpace):
    def __init__(self, space_id):
        super().__init__(space_id, 60)
        self.has_charger = True

class VIPParkingSpace(ParkingSpace):
    def __init__(self, space_id):
        super().__init__(space_id, 50)

class Node:
    def __init__(self, parking_space):
        self.parking_space = parking_space
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def add_space(self, parking_space):
        new_node = Node(parking_space)
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find_space(self):
        current = self.head
        while current:
            if not current.parking_space.is_occupied:
                return current.parking_space
            current = current.next
        return None

    def display_spaces(self):
        current = self.head
        while current:
            space = current.parking_space
            print(f"Space ID: {space.space_id}, Occupied: {space.is_occupied}, Cost: {space.cost} INR")
            current = current.next

class BSTNode:
    def __init__(self, space_id):
        self.space_id = space_id
        self.is_occupied = False
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None

    def insert(self, space_id):
        def _insert(root, space_id):
            if not root:
                return BSTNode(space_id)
            if space_id < root.space_id:
                root.left = _insert(root.left, space_id)
            else:
                root.right = _insert(root.right, space_id)
            return root
        self.root = _insert(self.root, space_id)

    def update_status(self, space_id, status):
        def _update(root):
            if not root:
                return
            if root.space_id == space_id:
                root.is_occupied = status
            elif space_id < root.space_id:
                _update(root.left)
            else:
                _update(root.right)
        _update(self.root)

    def display(self):
        def _display(root):
            if root:
                _display(root.left)
                status = "Occupied" if root.is_occupied else "Free"
                print(f"Slot {root.space_id} - {status}")
                _display(root.right)
        _display(self.root)

class ParkingLot:
    def __init__(self):
        self.parking_spaces = LinkedList()
        self.waiting_queue = []
        self.bst = BST()

    def add_space(self, space_id, space_type="regular"):
        if space_type == "ev":
            space = EVParkingSpace(space_id)
        elif space_type == "vip":
            space = VIPParkingSpace(space_id)
        else:
            space = ParkingSpace(space_id, 30)
        self.parking_spaces.add_space(space)
        self.bst.insert(space_id)

    def calculate_dynamic_price(self):
        occupied = 0
        total = 0
        current = self.parking_spaces.head
        while current:
            total += 1
            if current.parking_space.is_occupied:
                occupied += 1
            current = current.next

        occupancy_rate = occupied / total if total else 0
        multiplier = 1.5 if occupancy_rate > 0.5 else 1.0

        current = self.parking_spaces.head
        while current:
            space = current.parking_space
            space.cost = int(space.base_cost * multiplier)
            current = current.next

        print(f"--- Dynamic Pricing Multiplier: {multiplier} applied to all slot base prices ---")

    def allocate_space(self):
        space = self.parking_spaces.find_space()
        if space:
            space.occupy()
            self.bst.update_status(space.space_id, True)
            self.calculate_dynamic_price()
            return space.space_id
        else:
            print("No available spaces. Adding user to waiting queue.")
            self.waiting_queue.append("User")
            return -1

    def release_space(self, space_id):
        current = self.parking_spaces.head
        while current:
            if current.parking_space.space_id == space_id and current.parking_space.is_occupied:
                current.parking_space.vacate()
                self.bst.update_status(space_id, False)
                self.calculate_dynamic_price()
                print(f"Space {space_id} is now free.")
                return
            current = current.next
        print("Space not found or already free.")

    def update_slot_price(self, space_id, new_price):
        current = self.parking_spaces.head
        while current:
            if current.parking_space.space_id == space_id:
                current.parking_space.base_cost = new_price
                current.parking_space.cost = new_price  # will be adjusted by multiplier on next update
                print(f"Updated base price of space {space_id} to {new_price} INR")
                return
            current = current.next
        print("Space not found.")

    def display_parking_slots(self):
        print("--- Linked List View ---")
        self.parking_spaces.display_spaces()
        print("--- BST View ---")
        self.bst.display()

if __name__ == "__main__":
    system = ParkingLot()
    system.add_space(1)
    system.add_space(2)
    system.add_space(3)
    system.add_space(4)
    system.add_space(5)
    system.add_space(6, "ev")
    system.add_space(7, "vip")
    system.add_space(8)
    system.add_space(9, "ev")
    system.add_space(10, "vip")

    print("\nAvailable parking slots:")
    system.display_parking_slots()

    print("\nAllocating two parking slots...")
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    print("Allocated Slot:", system.allocate_space())
    

    print("\nUpdated parking slots:")
    system.display_parking_slots()

    print("\nReleasing space 1...")
    system.release_space(1)

    print("\nUpdating base price of space ID 2:")
    system.update_slot_price(2, 35)

    print("\nFinal state:")
    system.display_parking_slots()



Available parking slots:
--- Linked List View ---
Space ID: 1, Occupied: False, Cost: 30 INR
Space ID: 2, Occupied: False, Cost: 30 INR
Space ID: 3, Occupied: False, Cost: 30 INR
Space ID: 4, Occupied: False, Cost: 30 INR
Space ID: 5, Occupied: False, Cost: 30 INR
Space ID: 6, Occupied: False, Cost: 60 INR
Space ID: 7, Occupied: False, Cost: 50 INR
Space ID: 8, Occupied: False, Cost: 30 INR
Space ID: 9, Occupied: False, Cost: 60 INR
Space ID: 10, Occupied: False, Cost: 50 INR
--- BST View ---
Slot 1 - Free
Slot 2 - Free
Slot 3 - Free
Slot 4 - Free
Slot 5 - Free
Slot 6 - Free
Slot 7 - Free
Slot 8 - Free
Slot 9 - Free
Slot 10 - Free

Allocating two parking slots...
--- Dynamic Pricing Multiplier: 1.0 applied to all slot base prices ---
Allocated Slot: 1
--- Dynamic Pricing Multiplier: 1.0 applied to all slot base prices ---
Allocated Slot: 2
--- Dynamic Pricing Multiplier: 1.0 applied to all slot base prices ---
Allocated Slot: 3
--- Dynamic Pricing Multiplier: 1.0 applied to all slot b